In [3]:
DATA_DIR = 'input'

In [4]:
import os
from scipy.io import loadmat

# Load the camera calibration matrix data
calib = loadmat(os.path.join(DATA_DIR, 'dino_Ps.mat'))


In [5]:
import numpy as np

matrices = calib['P'][0]

In [6]:
import cv2 as cv

# Read the images in the input folder
IMAGES_DIR = os.path.join(DATA_DIR, 'images')

images = []
for file_name in os.listdir(IMAGES_DIR):
    image = cv.imread(os.path.join(IMAGES_DIR, file_name))
    images.append(image)

In [7]:
# Apply triangulation to the points

#images_3d = cv.triangulatePoints(np.array(images), matrices)

In [8]:
import ipywidgets as wg
import PIL.Image

def show_image_by_index(index: int) -> PIL.Image:
    rgb_image = cv.cvtColor(images[index], cv.COLOR_BGR2RGB)
    return PIL.Image.fromarray(rgb_image)

wg.interact(
    show_image_by_index,
    index=wg.IntSlider(min=0, max=len(images)-1, step=1));

interactive(children=(IntSlider(value=0, description='index', max=36), Output()), _dom_classes=('widget-intera…

In [9]:
# Crop Image black borders
images = [img[5:, :-30] for img in images]

In [10]:
import ipywidgets as wg
import PIL.Image

def show_image_by_index(index: int) -> PIL.Image:
    rgb_image = cv.cvtColor(images[index], cv.COLOR_BGR2RGB)
    return PIL.Image.fromarray(rgb_image)

wg.interact(
    show_image_by_index,
    index=wg.IntSlider(min=0, max=len(images)-1, step=1));

interactive(children=(IntSlider(value=0, description='index', max=36), Output()), _dom_classes=('widget-intera…

In [11]:
images_blur = [cv.GaussianBlur(image, (21, 21), 0) for image in images]

In [12]:
# TODO: replicated cell code

import ipywidgets as wg
import PIL.Image

def show_image_by_index_(index: int) -> PIL.Image:
    return PIL.Image.fromarray(cv.cvtColor(images_blur[index], cv.COLOR_BGR2RGB))

wg.interact(
    show_image_by_index_,
    index=wg.IntSlider(min=0, max=len(images_blur)-1, step=1));

interactive(children=(IntSlider(value=0, description='index', max=36), Output()), _dom_classes=('widget-intera…

In [13]:
images_hsv = [cv.cvtColor(image, cv.COLOR_BGR2LAB) for image in images_blur]
images_mask = [
    cv.threshold(img[..., 2], 0, 255, cv.THRESH_BINARY + cv.THRESH_OTSU)[1]
    for img in images_hsv]

In [14]:
# TODO: replicated cell code

import ipywidgets as wg
import PIL.Image

def show_image_by_index_(index: int) -> PIL.Image:
    return PIL.Image.fromarray(images_mask[index])

wg.interact(
    show_image_by_index_,
    index=wg.IntSlider(min=0, max=len(images)-1, step=1));

interactive(children=(IntSlider(value=0, description='index', max=36), Output()), _dom_classes=('widget-intera…

In [15]:
kernel = cv.getStructuringElement(cv.MORPH_ELLIPSE, (5, 5))
closing = [cv.morphologyEx(img, cv.MORPH_CLOSE, kernel) for img in images_mask]

In [16]:
# TODO: replicated cell code

import ipywidgets as wg
import PIL.Image

def show_image_by_index_(index: int) -> PIL.Image:
    return PIL.Image.fromarray(closing[index])

wg.interact(
    show_image_by_index_,
    index=wg.IntSlider(min=0, max=len(images)-1, step=1));

interactive(children=(IntSlider(value=0, description='index', max=36), Output()), _dom_classes=('widget-intera…

In [17]:
segmented_images = [cv.bitwise_and(img, img, mask=mask) for img, mask in zip(images, closing)]

In [18]:
#segmented_images

In [19]:
import ipywidgets as wg
import PIL.Image

def show_image_by_index_(index: int) -> PIL.Image:
    return PIL.Image.fromarray(cv.cvtColor(segmented_images[index], cv.COLOR_BGR2RGB))

wg.interact(
    show_image_by_index_,
    index=wg.IntSlider(min=0, max=len(images)-1, step=1));

interactive(children=(IntSlider(value=0, description='index', max=36), Output()), _dom_classes=('widget-intera…

In [20]:
'''import open3d as o3d

def carve_sides():
	print("creating voxel space...")
	voxel_grid = o3d.geometry.VoxelGrid.create_dense(
     	origin=[-1,-1,-1],
     	color=[1,0,1],
     	voxel_size=0.00325,
     	width=2,
     	height=2,
     	depth=2)
	print("created voxel space!")
	
	file_num = 1
	for img, params in zip(segmented_images, matrices):
		voxel_grid.carve_silhouette(img, params)
		print(file_num)
		file_num = file_num + 1
		
	o3d.visualization.draw_geometries([voxel_grid])
#carve_sides('./temple/','./temple/temple_par.txt', './temple/temple_ang.txt')''';

In [21]:
matrices[0]

array([[ 3.99235688e+00,  3.94176810e+01, -7.63289880e-01,
         3.95917551e+00],
       [-1.44302310e+01, -9.41441580e-01, -2.74509701e+01,
        -1.44294334e+01],
       [ 1.22492404e-02, -1.45746038e-04, -5.69307087e-04,
         1.22493587e-02]])

In [22]:
'''
import matplotlib.pyplot as plt

# Define the 3D point in homogeneous coordinate in order to multiply with the PPM 
_3d_point_homogeneous = np.array([[0.], [0.], [0.], [1.]])

# Multiply by the PPM obtaining the pixel in homogeneous coordinate
_2d_point_homogeneous = np.matmul(matrices[0], _3d_point_homogeneous)

# Divide by the third value to get the pixel coordinate
_2d_point = _2d_point_homogeneous / _2d_point_homogeneous[-1, 0]

print("2D point using PPM (Coordinates referred to the undistorted image): ", ( int(round(_2d_point[0,0])), int(round(_2d_point[1,0])) ))
# Draw the point
img2 = np.copy(segmented_images[0])
cv.circle(img2, (_2d_point[0], _2d_point[1]), 100, (255, 0, 0), 20)

# Crop away the black region obtained by undistorsion
#img2 = img2[y:y+h_2, x:x+w_2]

plt.figure(figsize=(10, 10))
plt.imshow(img2)
plt.show()''';

In [23]:
imgH, imgW, __ = segmented_images[0].shape

https://github.com/zinsmatt/SpaceCarving/blob/master/space_carving.py

https://github.com/zinsmatt/SpaceCarving/issues/7

In [24]:
'''#%%
# create voxel grid
s = 50 #120
# x, y, z = np.mgrid[:s, :s, :s]
pts = np.vstack((x.flatten(), y.flatten(), z.flatten())).astype(float)
pts = pts.T
nb_points_init = pts.shape[0]
xmax, ymax, zmax = np.max(pts, axis=0)
pts[:, 0] /= xmax
pts[:, 1] /= ymax
pts[:, 2] /= zmax
center = pts.mean(axis=0)
pts -= center
pts /= 5
pts[:, 2] -= 0.62
pts = np.vstack((pts.T, np.ones((1, nb_points_init))))''';

In [25]:
#pts.shape

(4, 125000)

In [26]:
'''print(np.unique(pts[0]))
print(np.unique(pts[1]))
print(np.unique(pts[2]))''';

In [27]:
'''np.mgrid[:s, :s, :s]''';

In [28]:
# Get the points of a voxel grid.

# Set the number of points in each dimension of the grid.
s = 120
# Get a coordinate grid of dimension (s x s x s).
x, y, z = np.meshgrid(np.arange(s), np.arange(s), np.arange(s))
# Flatten the coordinate arrays.
x, y, z = x.flatten(), y.flatten(), z.flatten()
# Scale the coordinate points to hardcoded values in the 3D space.
# The values are chosen such that the points of the grid enclose the dinosaur.
x = -.05 + (x - x.min()) * (.05 - (-.05)) / (x.max() - x.min())
y = -.1 + (y - y.min()) * (.04 - (-.1)) / (y.max() - y.min())
z = -.75 + (z - z.min()) * (-.5 - (-.75)) / (z.max() - z.min())
# Stack the coordinate arrays followed by a row of ones to get a matrix of
# homogeneous coordinates.
pts = np.vstack((x, y, z, np.ones(x.shape))).astype(float)

In [29]:
pts.shape

(4, 1728000)

In [37]:
img_with_voxels = []

for P, im in zip(matrices, closing):
    im = im.copy()
    uvs = P @ pts
    uvs /= uvs[-1, :]
    uvs = np.round(uvs).astype(int)
    im = cv.cvtColor(im, cv.COLOR_GRAY2BGRA)
    for uv in uvs.T:  # Iterate by column
        im = cv.circle(im, (uv[0], uv[1]), radius=1, color=(0, 0, 255, 50), thickness=-1)
    img_with_voxels.append(im)

In [38]:
import ipywidgets as wg
import PIL.Image

def show_image_by_index_(index: int) -> PIL.Image:
    return PIL.Image.fromarray(cv.cvtColor(img_with_voxels[index], cv.COLOR_BGR2RGBA))

wg.interact(
    show_image_by_index_,
    index=wg.IntSlider(min=0, max=len(img_with_voxels)-1, step=1));

interactive(children=(IntSlider(value=0, description='index', max=35), Output()), _dom_classes=('widget-intera…

In [40]:
filled = []
for P, im in zip(matrices, closing):
    # Compute the pixel coordinates of the points
    uvs = P @ pts
    # Divide by the third value to get the pixel coordinate
    uvs /= uvs[-1, :]
    # Convert the points to integer
    uvs = np.round(uvs).astype(int)
    # 
    x_good = np.logical_and(uvs[0, :] >= 0, uvs[0, :] < imgW)
    y_good = np.logical_and(uvs[1, :] >= 0, uvs[1, :] < imgH)
    good = np.logical_and(x_good, y_good)
    indices = np.where(good)[0]
    fill = np.zeros(uvs.shape[1])
    sub_uvs = uvs[:-1, indices]
    res = im[sub_uvs[1, :], sub_uvs[0, :]]
    fill[indices] = res 

    filled.append(fill)
filled = np.vstack(filled)

# the occupancy is computed as the number of camera in which the point "seems" not empty
occupancy = np.sum(filled, axis=0)

# Select occupied voxels
pts = pts.T
good_points = pts[occupancy > 4, :]

In [41]:
good_points = pts[occupancy > 8_000, :]
good_points.shape

(86732, 4)

In [42]:
img_with_3D = []

for P, im in zip(matrices, closing):
    im = np.zeros_like(im)
    uvs = P @ good_points.T
    uvs /= uvs[-1, :]
    uvs = np.round(uvs).astype(int)
    im = cv.cvtColor(im, cv.COLOR_GRAY2BGRA)
    for uv in uvs.T:  # Iterate by column
        im = cv.circle(im, (uv[0], uv[1]), radius=1, color=(0, 0, 255, 30), thickness=-1)
    img_with_3D.append(im)

In [43]:
import ipywidgets as wg
import PIL.Image

def show_image_by_index_(index: int) -> PIL.Image:
    return PIL.Image.fromarray(cv.cvtColor(img_with_3D[index], cv.COLOR_BGR2RGBA))

wg.interact(
    show_image_by_index_,
    index=wg.IntSlider(min=0, max=len(img_with_3D)-1, step=1));

interactive(children=(IntSlider(value=0, description='index', max=35), Output()), _dom_classes=('widget-intera…

In [96]:
# Get the s x s x s coordinates of the voxel grid. Map the coordinates
# as the ones of paraview (x -> y; y -> z; z -> x).

y = pts[:s*s:s, 0]
z = pts[::s*s, 1]
x = pts[:s, 2]

In [97]:
print(x.shape, y.shape, z.shape)

(120,) (120,) (120,)


In [99]:
# Save point cloud with occupancy scalar 
filename = "shape.txt"
with open(filename, "w") as fout:
    fout.write("x,y,z,occ\n")
    for occ, p in zip(occupancy, pts[:, :3]):
        fout.write(",".join(p.astype(str)) + "," + str(occ) + "\n")

# Save as rectilinear grid (this enables paraview to display its iso-volume as a mesh)
import vtk

xCoords = vtk.vtkFloatArray()
#x = pts[::s*s, 0]
#y = pts[:s*s:s, 1]
#z = pts[:s, 2]
for i in x:
    xCoords.InsertNextValue(i)
yCoords = vtk.vtkFloatArray()
for i in y:
    yCoords.InsertNextValue(i)
zCoords = vtk.vtkFloatArray()
for i in z:
    zCoords.InsertNextValue(i)
values = vtk.vtkFloatArray()
for i in occupancy:
    values.InsertNextValue(i)
rgrid = vtk.vtkRectilinearGrid()
rgrid.SetDimensions(len(x), len(y), len(z))
rgrid.SetXCoordinates(xCoords)
rgrid.SetYCoordinates(yCoords)
rgrid.SetZCoordinates(zCoords)
rgrid.GetPointData().SetScalars(values)

writer = vtk.vtkXMLRectilinearGridWriter()
writer.SetFileName("shape.vtr")
writer.SetInputData(rgrid)
writer.Write()

1

: 